In [0]:
# from mxnet import autograd, gluon, init, nd
# from mxnet.contrib import text
# from mxnet.gluon import data as gdata, loss as gloss, nn, rnn, utils as gutils
# from tqdm import tqdm_notebook

# import mxnet as mx
import collections
import os
import random
import tarfile
import time
#!pip install numpy==1.16.1
#!pip install mxnet-cu100mkl -- google colab에서 mxnet사용하려면 해당 버전 다운 필요
#!pip install gensim -- 넘파이 1.16버전 사용해야함
import numpy as np
 

In [0]:
!pip3 install http://download.pytorch.org/whl/cu80/torch-0.3.0.post4-cp36-cp36m-linux_x86_64.whl
!pip3 install torchvision

In [0]:
!{sys.executable} -m pip install torchtext==0.2.3

     |████████████████████████████████| 51kB 2.5MB/s 
  Stored in directory: /root/.cache/pip/wheels/42/a6/f4/b267328bde6bb680094a0c173e8e5627ccc99543abded97204
Successfully built torchtext
  Found existing installation: torchtext 0.3.1
    Uninstalling torchtext-0.3.1:
      Successfully uninstalled torchtext-0.3.1


In [0]:
import os
import sys
import torch
from torch.nn import functional as F
import numpy as np
from torchtext import data
from torchtext import datasets
from torchtext.vocab import Vectors, GloVe
from torchtext.data import TabularDataset

In [0]:
from torchtext.data import Iterator

In [0]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2018 NVIDIA Corporation
Built on Sat_Aug_25_21:08:01_CDT_2018
Cuda compilation tools, release 10.0, V10.0.130


#   import library 


In [0]:
import os
import sys
import numpy as np

import torch
import torch.nn as nn
from torch.nn import functional as F
from torchtext import data
from torchtext import datasets
from torchtext.vocab import Vectors, GloVe
from torch.autograd import Variable 

# Load_dataset 

In [0]:
def load_dataset(test_sen=None):

    """
    tokenizer : Breaks sentences into a list of words. If sequential=False, no tokenization is applied
    Field : A class that stores information about the way of preprocessing
    fix_length : An important property of TorchText is that we can let the input to be variable length, and TorchText will
                 dynamically pad each sequence to the longest sequence in that "batch". But here we are using fi_length which
                 will pad each sequence to have a fix length of 200.
                 
    build_vocab : It will first make a vocabulary or dictionary mapping all the unique words present in the train_data to an
                  idx and then after it will use GloVe word embedding to map the index to the corresponding word embedding.
                  
    vocab.vectors : This returns a torch tensor of shape (vocab_size x embedding_dim) containing the pre-trained word embeddings.
    BucketIterator : Defines an iterator that batches examples of similar lengths together to minimize the amount of padding needed.
    
    """
    
    tokenize = lambda x: x.split()
    TEXT = data.Field(sequential=True, tokenize=tokenize, lower=True, include_lengths=True, batch_first=True, fix_length=200)
    LABEL = data.Field(sequential=False, use_vocab=False,   
              preprocessing = lambda x: 1 if x == "pos" else 0,  
              batch_first=True)
    train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)
    TEXT.build_vocab(train_data, vectors=GloVe(name='6B', dim=300))
    LABEL.build_vocab(train_data)

    word_embeddings = TEXT.vocab.vectors
    print ("Length of Text Vocabulary: " + str(len(TEXT.vocab)))
    print ("Vector size of Text Vocabulary: ", TEXT.vocab.vectors.size())
    print ("Label Length: " + str(len(LABEL.vocab)))

    train_data, valid_data = train_data.split() # Further splitting of training_data to create new training_data & validation_data
    train_iter, valid_iter, test_iter = data.BucketIterator.splits((train_data, valid_data, test_data), batch_size=32, sort_key=lambda x: len(x.text), repeat=False, shuffle=True)

    '''Alternatively we can also use the default configurations'''
    # train_iter, test_iter = datasets.IMDB.iters(batch_size=32)

    vocab_size = len(TEXT.vocab)

    return TEXT, vocab_size, word_embeddings, train_iter, valid_iter, test_iter

In [3]:
TEXT, vocab_size, word_embeddings, train_iter, valid_iter, test_iter = load_dataset()


aclImdb_v1.tar.gz:   0%|          | 0.00/84.1M [00:00<?, ?B/s]

downloading aclImdb_v1.tar.gz


aclImdb_v1.tar.gz: 100%|██████████| 84.1M/84.1M [00:02<00:00, 38.9MB/s]
.vector_cache/glove.6B.zip: 862MB [01:27, 9.83MB/s]                           
100%|█████████▉| 399561/400000 [00:38<00:00, 10392.64it/s]

Length of Text Vocabulary: 251639
Vector size of Text Vocabulary:  torch.Size([251639, 300])
Label Length: 3


## one batch example

In [11]:
for i,train in enumerate(train_iter):
  if i == 0:
    print(train.text)
    print(train.label)
    print(len(train)) 
    aa = train

(tensor([[  1543,   3067,      7,  ...,      1,      1,      1],
        [  3823,    113,     12,  ...,      1,      1,      1],
        [   896,     47,      2,  ...,      1,      1,      1],
        ...,
        [106556,  17922,   3743,  ...,      1,      1,      1],
        [    12,      7,     23,  ...,  40733,   1300,      6],
        [  1551,     94,    855,  ...,     44,    168,    531]]), tensor([178, 126, 145,  44, 143, 101, 132, 162, 200, 200, 133, 200, 193, 200,
         52, 146, 200, 114, 162, 200,  92, 200,  70, 184, 152, 141, 200, 200,
        174, 169, 200, 200]))
tensor([1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0,
        1, 1, 1, 1, 1, 0, 1, 1])
32


In [34]:
len(TEXT.vocab.itos)

251639

# Modeling

In [0]:
learning_rate = 2e-5
batch_size = 32
output_size = 2
hidden_size = 256
embedding_length = 300


In [0]:
def clip_gradient(model, clip_value):
    params = list(filter(lambda p: p.grad is not None, model.parameters()))
    for p in params:
        p.grad.data.clamp_(-clip_value, clip_value)
    
def train_model(model, train_iter, epoch):
    total_epoch_loss = 0
    total_epoch_acc = 0
    model.cuda()
    optim = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()))
    steps = 0
    model.train()
    for idx, batch in enumerate(train_iter):
        text = batch.text[0]
        target = batch.label
        target = torch.autograd.Variable(target).long()
        if torch.cuda.is_available():
            text = text.cuda()
            target = target.cuda()
        if (text.size()[0] is not 32):# One of the batch returned by BucketIterator has length different than 32.
            continue
        optim.zero_grad()
        prediction,att= model(text)
        loss = loss_fn(prediction, target)
        num_corrects = (torch.max(prediction, 1)[1].view(target.size()).data == target.data).float().sum()
        acc = 100.0 * num_corrects/len(batch)
        loss.backward()
        clip_gradient(model, 1e-1)
        optim.step()
        steps += 1
        
        if steps % 100 == 0:
            print (f'Epoch: {epoch+1}, Idx: {idx+1}, Training Loss: {loss.item():.4f}, Training Accuracy: {acc.item(): .2f}%')
        
        total_epoch_loss += loss.item()
        total_epoch_acc += acc.item()
        
    return total_epoch_loss/len(train_iter), total_epoch_acc/len(train_iter)

def eval_model(model, val_iter):
    total_epoch_loss = 0
    total_epoch_acc = 0
    model.eval()
    with torch.no_grad():
        for idx, batch in enumerate(val_iter):
            text = batch.text[0]
            if (text.size()[0] is not 32):
                continue
            target = batch.label
            target = torch.autograd.Variable(target).long()
            if torch.cuda.is_available():
                text = text.cuda()
                target = target.cuda()
            prediction,att = model(text)
            loss = loss_fn(prediction, target)
            num_corrects = (torch.max(prediction, 1)[1].view(target.size()).data == target.data).sum()
            acc = 100.0 * num_corrects/len(batch)
            total_epoch_loss += loss.item()
            total_epoch_acc += acc.item()

    return total_epoch_loss/len(val_iter), total_epoch_acc/len(val_iter),att
	


In [0]:
class AttentionModel(torch.nn.Module):
	def __init__(self, batch_size, output_size, hidden_size, vocab_size, embedding_length, weights):
		super(AttentionModel, self).__init__()
		
		"""
		Arguments
		---------
		batch_size : Size of the batch which is same as the batch_size of the data returned by the TorchText BucketIterator
		output_size : 2 = (pos, neg)
		hidden_sie : Size of the hidden_state of the LSTM
		vocab_size : Size of the vocabulary containing unique words
		embedding_length : Embeddding dimension of GloVe word embeddings
		weights : Pre-trained GloVe word_embeddings which we will use to create our word_embedding look-up table 
		
		--------
		
		"""
		
		self.batch_size = batch_size
		self.output_size = output_size
		self.hidden_size = hidden_size
		self.vocab_size = vocab_size
		self.embedding_length = embedding_length
		
		self.word_embeddings = nn.Embedding(vocab_size, embedding_length)
		self.word_embeddings.weights = nn.Parameter(weights, requires_grad=False)
		self.lstm = nn.LSTM(embedding_length, hidden_size)
		self.label = nn.Linear(hidden_size, output_size)
		#self.attn_fc_layer = nn.Linear()
		
	def attention_net(self, lstm_output, final_state):

		""" 
		Now we will incorporate Attention mechanism in our LSTM model. In this new model, we will use attention to compute soft alignment score corresponding
		between each of the hidden_state and the last hidden_state of the LSTM. We will be using torch.bmm for the batch matrix multiplication.
		
		Arguments
		---------
		
		lstm_output : Final output of the LSTM which contains hidden layer outputs for each sequence.
		final_state : Final time-step hidden state (h_n) of the LSTM
		
		---------
		
		Returns : It performs attention mechanism by first computing weights for each of the sequence present in lstm_output and and then finally computing the
				  new hidden state.
				  
		Tensor Size :
					hidden.size() = (batch_size, hidden_size)
					attn_weights.size() = (batch_size, num_seq)
					soft_attn_weights.size() = (batch_size, num_seq)
					new_hidden_state.size() = (batch_size, hidden_size)
					  
		"""
		
		hidden = final_state.squeeze(0)
		attn_weights = torch.bmm(lstm_output, hidden.unsqueeze(2)).squeeze(2)
		soft_attn_weights = F.softmax(attn_weights, 1)
		new_hidden_state = torch.bmm(lstm_output.transpose(1, 2), soft_attn_weights.unsqueeze(2)).squeeze(2)
		
		return new_hidden_state,soft_attn_weights
	
	def forward(self, input_sentences, batch_size=None):
	
		""" 
		Parameters
		----------
		input_sentence: input_sentence of shape = (batch_size, num_sequences)
		batch_size : default = None. Used only for prediction on a single sentence after training (batch_size = 1)
		
		Returns
		-------
		Output of the linear layer containing logits for pos & neg class which receives its input as the new_hidden_state which is basically the output of the Attention network.
		final_output.shape = (batch_size, output_size)
		
		"""
		
		input = self.word_embeddings(input_sentences)
		input = input.permute(1, 0, 2)
		if batch_size is None:
			h_0 = Variable(torch.zeros(1, self.batch_size, self.hidden_size).cuda())
			c_0 = Variable(torch.zeros(1, self.batch_size, self.hidden_size).cuda())
		else:
			h_0 = Variable(torch.zeros(1, batch_size, self.hidden_size).cuda())
			c_0 = Variable(torch.zeros(1, batch_size, self.hidden_size).cuda())
			
		output, (final_hidden_state, final_cell_state) = self.lstm(input, (h_0, c_0)) # final_hidden_state.size() = (1, batch_size, hidden_size) 
		output = output.permute(1, 0, 2) # output.size() = (batch_size, num_seq, hidden_size)
		
		attn_output,attention_weight = self.attention_net(output, final_hidden_state)
		#print(attention_weight)
		logits = self.label(attn_output)
		
		return logits,attention_weight

In [0]:
model = AttentionModel(batch_size, output_size, hidden_size, vocab_size, embedding_length, word_embeddings)
loss_fn = F.cross_entropy

In [40]:
for epoch in range(10):
    train_loss, train_acc = train_model(model, train_iter, epoch)
    val_loss, val_acc,_ = eval_model(model, valid_iter)
    
    print(f'Epoch: {epoch+1:02}, Train Loss: {train_loss:.3f}, Train Acc: {train_acc:.2f}%, Val. Loss: {val_loss:3f}, Val. Acc: {val_acc:.2f}%')
    
test_loss, test_acc,_ = eval_model(model, test_iter)
print(f'Test Loss: {test_loss:.3f}, Test Acc: {test_acc:.2f}%')

''' Let us now predict the sentiment on a single sentence just for the testing purpose. '''
test_sen1 = "This is one of the best creation of Nolan. I can say, it's his magnum opus. Loved the soundtrack and especially those creative dialogues."
test_sen2 = "Ohh, such a ridiculous movie. Not gonna recommend it to anyone. Complete waste of time and money."

test_sen1 = TEXT.preprocess(test_sen1)
test_sen1 = [[TEXT.vocab.stoi[x] for x in test_sen1]]

test_sen2 = TEXT.preprocess(test_sen2)
test_sen2 = [[TEXT.vocab.stoi[x] for x in test_sen2]]

test_sen = np.asarray(test_sen1)
test_sen = torch.LongTensor(test_sen)
test_tensor = Variable(test_sen, volatile=True)
test_tensor = test_tensor.cuda()


Epoch: 1, Idx: 100, Training Loss: 0.6885, Training Accuracy:  56.25%
Epoch: 1, Idx: 200, Training Loss: 0.6958, Training Accuracy:  53.12%
Epoch: 1, Idx: 300, Training Loss: 0.6887, Training Accuracy:  62.50%
Epoch: 1, Idx: 400, Training Loss: 0.6290, Training Accuracy:  68.75%
Epoch: 1, Idx: 500, Training Loss: 0.6521, Training Accuracy:  65.62%
Epoch: 01, Train Loss: 0.675, Train Acc: 57.56%, Val. Loss: 0.583864, Val. Acc: 69.72%
Epoch: 2, Idx: 100, Training Loss: 0.3394, Training Accuracy:  84.38%
Epoch: 2, Idx: 200, Training Loss: 0.5695, Training Accuracy:  75.00%
Epoch: 2, Idx: 300, Training Loss: 0.4713, Training Accuracy:  71.88%
Epoch: 2, Idx: 400, Training Loss: 0.1711, Training Accuracy:  96.88%
Epoch: 2, Idx: 500, Training Loss: 0.3395, Training Accuracy:  84.38%
Epoch: 02, Train Loss: 0.417, Train Acc: 81.21%, Val. Loss: 0.428006, Val. Acc: 81.69%
Epoch: 3, Idx: 100, Training Loss: 0.1942, Training Accuracy:  84.38%
Epoch: 3, Idx: 200, Training Loss: 0.2531, Training Accu

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:22: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.


In [41]:
test_sen1 = "This is one of the best creation of Nolan. I can say, it's his magnum opus. Loved the soundtrack and especially those creative dialogues."
test_sen1 = TEXT.preprocess(test_sen1)
test_sen1 = [[TEXT.vocab.stoi[x] for x in test_sen1]]

test_sen = np.asarray(test_sen1)
test_sen = torch.LongTensor(test_sen)
test_tensor = Variable(test_sen, volatile=True)
test_tensor = test_tensor.cuda()


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  if sys.path[0] == '':


In [0]:
def check_result(sentence):
  test_sen2 = sentence
  test_sen2 = TEXT.preprocess(test_sen2)
  tmp_sen2 = test_sen2.copy()
  test_sen2 = [[TEXT.vocab.stoi[x] for x in test_sen2]]

  test_sen = np.asarray(test_sen2)
  test_sen = torch.LongTensor(test_sen)
  test_tensor = Variable(test_sen)
  test_tensor = test_tensor.cuda()

  output,att = model(test_tensor, 1)
  out = F.softmax(output, 1)

  reason = tmp_sen2[torch.max(att,1).indices[0].item()]
  if (torch.argmax(out[0]) == 1):
      print ("Sentiment: Positive --- Reason: %s"  % reason) 
  else:
      print ("Sentiment: Negative --- Reason: %s"  % reason)

In [94]:
sentence = "Ohh, such a ridiculous movie. Not gonna recommend it to anyone. Complete waste of time and money."
check_result(sentence)

Sentiment: Negative --- Reason: waste


In [95]:
sentence = "This is one of the best creation of Nolan. I can say, it's his magnum opus. Loved the soundtrack and especially those creative dialogues."
check_result(sentence)

Sentiment: Positive --- Reason: dialogues.
